<a href="https://colab.research.google.com/github/Jackelyneg/covid-19-image-classification/blob/main/flaskapp_Xray_symptoms_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok
!pip install flask-bootstrap

     |████████████████████████████████| 456 kB 9.8 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460124 sha256=285e495a592ff2ace3216dfe1902fa0d1118f3e86e97575099ba0f45a69a59b8
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3943 sha256=e3f7abddcbad9d749bb16f53f6d4b0b0020e39e27c7b601ee81211372f7a4991
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built flask-bootstrap visitor


In [2]:
from flask_ngrok import run_with_ngrok
import numpy as np
import tensorflow as tf
from flask import Flask,jsonify,make_response,request,render_template,flash,redirect,url_for
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
import pickle
import os

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
UPLOAD_FOLDER = '/content/drive/MyDrive/Colab Notebooks/project4/static'
ALLOWED_EXTENSIONS = {'jpg','jpeg'}


In [6]:



from flask import Flask, request, jsonify, render_template,url_for
import pickle
import folium
from math import cos, asin, sqrt, pi
import pandas as pd
import folium.plugins
folium.plugins.MarkerCluster()

from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder="/content/drive/MyDrive/Colab Notebooks/project4/templates",static_folder="/content/drive/MyDrive/Colab Notebooks/project4/static" )
app.config['SESSION_TYPE'] = 'memcached'
app.config['SECRET_KEY'] = 'super secret key'

run_with_ngrok(app)
# @app.route("/")
# def home():
#   return render_template('index.html')


app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    print('.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS)
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
@app.route("/")
def home():
  return render_template('index.html')

@app.route('/page2', methods=['GET', 'POST'])
def upload_file():

    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            predictions=download_file(name=filename)
            

            full_filename=url_for('static',filename = filename)

            print(full_filename)
            return render_template("xray.html",predictions_text=predictions,filename=full_filename) 
            #return redirect(url_for('download_file', name=filename))
    return render_template("xray.html")       
       


from flask import send_from_directory

@app.route('/uploads/<name>')
def download_file(name):
      
     return xraypred(name)
#@app.route("/getxraypredictions")
def xraypred(fileName):

  model = load_model("/content/drive/MyDrive/project4data/mymodel.h5")
  from tensorflow.keras.preprocessing import image
  filepath="/content/drive/MyDrive/Colab Notebooks/project4/static"+"/"+fileName
  print(filepath)
 
  image_size = (256, 256,1)
  im = image.load_img(filepath, target_size=image_size, color_mode="grayscale")
  
  from tensorflow.keras.preprocessing.image import img_to_array

  image = img_to_array(im)
  img = image.flatten().reshape( -1,256, 256,1)
  #plt.imshow(img.reshape(256, 256))
  img = 1 - img
  #plt.imshow(img.reshape(256, 256), cmap=plt.cm.Greys)
  predictions = model.predict(img)
  predicted= np.argmax(predictions,axis=1)
  if predicted ==0:
    predicted_string="The person has Covid"
  if predicted ==1:
    predicted_string="the person does not have Covid"
  #print(predicted_string)
  #predictions_string=np.array_str(predicted)
  
    
  return (predicted_string)
  

model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/project4/model2.pkl", 'rb'))  # file path may be different

@app.route('/symptoms')
def symptoms():
    return render_template('symptoms1.html')
@app.route('/predict', methods=['POST'])
def predict():
    int_features = [int(x) for x in request.form.values()]
    final_features=[np.array(int_features)]
    prediction=model.predict(final_features)
    output=round(prediction[0],2)
    if prediction == 1:
        pred = "Your symptoms show Covid detection, please consult a Doctor."
    else: # prediction == 0:
        pred = "You don't have Covid."
    output = pred
    return render_template('symptoms1.html', prediction_text='COVID Results {}'.format(output))


@app.route('/map',methods=['GET','POST'])
def drawmap():
  zipcode=request.form.get('zipcode')
  
  data=pd.read_csv("/content/drive/MyDrive/data.csv")
  hosp_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/testMap/hospital_zip.csv')
  hosp_df=(hosp_df[["hospital_name", "address", "city", "state", "zip", "LAT", "LNG"]])
  hosp_df['zip'] = hosp_df['zip'].apply(lambda x: '{0:0>5}'.format(x))
  
  for i, row in hosp_df.iterrows():
   
    if zipcode == row['zip']:
        lat2=row['LAT']
        long2=row['LNG']



  p=pi/180
  results=[]
  zips = []
  lats = []
  longs = []
  hospitals = []
  addresses= []
  cities = []
  states = []

  for i, row in hosp_df.iterrows():
      lat1=row['LAT']
      long1=row['LNG']
      hospital=hosp_df['hospital_name'][i]
      address=hosp_df['address'][i]
      city=hosp_df['city'][i]
      state=hosp_df['state'][i]
      zip_code=row['zip']
      a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((long2-long1)*p))/2
      result=12742*asin(sqrt(a))
      results.append(result)
      zips.append(zip_code)
      lats.append(lat1)
      longs.append(long1)
      hospitals.append(hospital)
      #print(hospitals)
      addresses.append(address)
      cities.append(city)
      states.append(state)
    
  df = pd.DataFrame(list(zip(results, zips, lats, longs, hospitals, addresses, cities, states)),
               columns =['distance', 'zip_codes', 'latitude', 'longitude', 'hospital', 'address', 'city', 'state'])
  fifteen_miles = df[df['distance']<= 15]
  import folium.plugins
  

  #Base Map
  usa = [37.090240, -95.712891]
  Map_object = folium.Map(location=usa, zoom_start=4)
  print(Map_object)
  
  for i in range(0, len(fifteen_miles)):
    folium.Marker([fifteen_miles.iloc[i]["latitude"], fifteen_miles.iloc[i]["longitude"]], popup=fifteen_miles.iloc[i]["hospital"]).add_to(Map_object)

  folium.plugins.MarkerCluster()
  folium.plugins.marker_cluster.MarkerCluster
  
  marker_cluster = MarkerCluster().add_to(Map_object)
  #marker_cluster = folium.plugins.MarkerCluster()("hopitals").add_to(Map)

# # Load locally stored colchester public toilets data

  for i in range(0, len(fifteen_miles)):
    folium.Marker([fifteen_miles.iloc[i]["latitude"], fifteen_miles.iloc[i]["longitude"]], popup=fifteen_miles.iloc[i]["zip_codes"]).add_to(marker_cluster)
  

  print(Map_object._repr_html_())
  return Map_object._repr_html_()





    

  
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.3 when using version 0.22.2.post1. This might lead to break

 * Running on http://2c03-34-125-101-184.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Sep/2021 04:29:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 04:29:11] "GET /content/drive/MyDrive/Colab%20Notebooks/project4/assets/js/markerclusterer_compiled.js HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2021 04:29:11] "GET /content/drive/MyDrive/Colab%20Notebooks/project4/assets/js/markerclusterer_compiled.js HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2021 04:29:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2021 04:29:14] "GET /symptoms HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 04:29:14] "GET /static/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2021 04:29:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 04:29:23] "GET /static/css/style.css HTTP/1.1" 404 -
